In [1]:
import geopandas as gpd
import time
import os
import matplotlib.pyplot as plt
import json
import requests
from osgeo import gdal
from matplotlib import pyplot
import rasterio
import rasterio.mask
from xml.dom.minidom import parse
import numpy
import pandas as pd
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
import os
import requests
PLANET_API_KEY = "17de9ecc82734c1caf0ef0da1bf90d97"
import datetime


In [2]:
# Set filepath
fp = "shapes/trial-design-n-2021.shp"
# Read file using gpd.read_file()
trial_design = gpd.read_file(fp)
#str(data.iloc[0]['geometry'])
# check the data
#gpd.GeoSeries([data.iloc[0]['geometry']])
print(len(trial_design.index))
trial_design.head(2)

140


,geometry
0,"POLYGON ((-89.06084 39.72437, -89.06084 39.724..."
1,"POLYGON ((-89.06172 39.72437, -89.06172 39.724..."


In [3]:
df = pd.read_csv('data/res.csv')
print(len(df.index))
df.head(3)
ids = [v.id for k,v in df.iterrows()]

34


In [4]:
for geom in trial_design.iterrows():
    print(geom[1])
    break

geometry    POLYGON ((-89.06083529248441 39.72436997827738...
Name: 0, dtype: object


In [12]:
def calc_nvdi(image_path, xml_path, out_path):
    my_image = rasterio.open(image_path)
    band_red = my_image.read(3)
    band_nir = my_image.read(4)
    xmldoc = parse(xml_path)
    nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")
    # XML parser refers to bands by numbers 1-4
    coeffs = {}
    for node in nodes:
        bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
        if bn in ['1', '2', '3', '4']:
            i = int(bn)
            value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
            coeffs[i] = float(value)

    # Multiply by corresponding coefficients
    band_red = band_red * coeffs[3]
    band_nir = band_nir * coeffs[4]
    # Allow division by zero
    numpy.seterr(divide='ignore', invalid='ignore')

    # Calculate NDVI
    ndvi = (band_nir.astype(float) - band_red.astype(float)) / (band_nir + band_red)

    # Set spatial characteristics of the output object to mirror the input
    kwargs = src.meta
    kwargs.update(
        dtype=rasterio.float32,
        count = 1)
    with rasterio.open(out_path, 'w', **kwargs) as dst:
        dst.write_band(1, ndvi.astype(rasterio.float32))
#    plt.imsave(f"data/imgs/{id_}_subarea_ndvi_cmap.png", ndvi, cmap=plt.cm.summer)
    return my_image

In [ ]:
for id_ in ids:
    image_path = f"data/imgs/{id_}_subarea.tif"
    out_path = f"data/nvdi/{id_}.tif"
    xml_path = parse(f"data/imgs/{id_}_subarea.xml")
    for p in [image_path,xml_path,out_path]:
        if not os.path.exists(p):
            os.makedirs(p)
    src = calc_nvdi(image_path,xml_path,out_path)
    #src = rasterio.open()
    #outpath = f"data/cropped/{id_}"
    if not os.path.exists(outpath):
        #Create a new directory because it does not exist 
        os.makedirs(outpath)
    i = 0
    for geom in trial_design.iterrows():
        out_image, out_transform = rasterio.mask.mask(src, geom[1], crop=True)
        out_meta = src.meta
        out_meta.update(
            {
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform,
            }
        )
        
        filename = f"{outpath}/{i}.tiff"
        #array = src.read(1)
        
        with rasterio.open(filename, "w", **out_meta) as dest:
            dest.write(out_image)
        i += 1
        new_plot_image = rasterio.open(filename)
        
        band_red = new_plot_image.read(3)
        band_nir = new_plot_image.read(4)


        nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")

        # XML parser refers to bands by numbers 1-4
        coeffs = {}
        for node in nodes:
            bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
            if bn in ['1', '2', '3', '4']:
                i = int(bn)
                value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
                coeffs[i] = float(value)

        # Multiply by corresponding coefficients
        band_red = band_red * coeffs[3]
        band_nir = band_nir * coeffs[4]
        # Allow division by zero
        numpy.seterr(divide='ignore', invalid='ignore')

        # Calculate NDVI
        ndvi = (band_nir.astype(float) - band_red.astype(float)) / (band_nir + band_red)

        # Set spatial characteristics of the output object to mirror the input
        kwargs = src.meta
        kwargs.update(
            dtype=rasterio.float32,
            count = 1)
        #print(ndvi)
        # Create the file
        #with rasterio.open(f"data/imgs/{id_}_subarea_ndvi.tif", 'w', **kwargs) as dst:
        #        dst.write_band(1, ndvi.astype(rasterio.float32))
        #plt.imsave(f"data/imgs/{id_}_subarea_ndvi_cmap.png", ndvi, cmap=plt.cm.summer)


In [ ]:
for id_ in ids:
    src = rasterio.open(f"data/imgs/{id_}_subarea.tif")
    outpath = f"data/cropped/{id_}"
    if not os.path.exists(outpath):
        #Create a new directory because it does not exist 
        os.makedirs(outpath)
    i = 0
    for geom in trial_design.iterrows():
        out_image, out_transform = rasterio.mask.mask(src, geom[1], crop=True)
        out_meta = src.meta
        out_meta.update(
            {
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform,
            }
        )
        
        filename = f"{outpath}/{i}.tiff"
        #array = src.read(1)
        
        with rasterio.open(filename, "w", **out_meta) as dest:
            dest.write(out_image)
        i += 1
        new_plot_image = rasterio.open(filename)
        
        band_red = new_plot_image.read(3)
        band_nir = new_plot_image.read(4)


        xmldoc = parse(f"data/imgs/{id_}_subarea.xml")
        nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")

        # XML parser refers to bands by numbers 1-4
        coeffs = {}
        for node in nodes:
            bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
            if bn in ['1', '2', '3', '4']:
                i = int(bn)
                value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
                coeffs[i] = float(value)

        # Multiply by corresponding coefficients
        band_red = band_red * coeffs[3]
        band_nir = band_nir * coeffs[4]
        # Allow division by zero
        numpy.seterr(divide='ignore', invalid='ignore')

        # Calculate NDVI
        ndvi = (band_nir.astype(float) - band_red.astype(float)) / (band_nir + band_red)

        # Set spatial characteristics of the output object to mirror the input
        kwargs = src.meta
        kwargs.update(
            dtype=rasterio.float32,
            count = 1)
        #print(ndvi)
        # Create the file
        #with rasterio.open(f"data/imgs/{id_}_subarea_ndvi.tif", 'w', **kwargs) as dst:
        #        dst.write_band(1, ndvi.astype(rasterio.float32))
        #plt.imsave(f"data/imgs/{id_}_subarea_ndvi_cmap.png", ndvi, cmap=plt.cm.summer)


In [ ]:
pyplot.imshow(array, cmap='pink')
pyplot.show()  

In [7]:
filename

'data/cropped/20210822_164358_51_2407/0.tiff'

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,  20,  16,   0],
       [142, 100,  75, ...,  16,  14,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)